# Execute Shell command
## Pre-Processing

In [1]:
# nuclio: ignore
import nuclio

Define the MLRun environment

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/mlrun"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/mlrun'


## Function

In [3]:
from mlrun import get_or_create_ctx
from kubernetes import config, client
from kubernetes.stream import stream

In [4]:
def handler(context,cmd,shell_pod_name = 'shell'):
    context.logger.info("Starting to cmd")    
    context.logger.info("Running command " + cmd)
    cli = K8SClient(context.logger)
    cli.exec_shell_cmd(cmd, shell_pod_name)

class K8SClient(object):

    def __init__(self, logger, namespace='default-tenant', config_file=None):
        self.namespace = namespace
        self.logger = logger
        self._init_k8s_config(config_file)
        self.v1api = client.CoreV1Api()

    def _init_k8s_config(self, config_file):
        try:
            config.load_incluster_config()
            self.logger.info('using in-cluster config.')
        except Exception:
            try:
                config.load_kube_config(config_file)
                self.logger.info('using local kubernetes config.')
            except Exception:
                raise RuntimeError(
                    'cannot find local kubernetes config file,'
                    ' place it in ~/.kube/config or specify it in '
                    'KUBECONFIG env var')

    def get_shell_pod_name(self, shell_pod_name):
        shell_pod = self.v1api.list_namespaced_pod(namespace=self.namespace)
        for i in shell_pod.items:
            if shell_pod_name in i.metadata.name:
                self.logger.info("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))
                shell_name = i.metadata.name
                break
        return shell_name

    def exec_shell_cmd(self, cmd, shell_pod_name):
        shell_name = self.get_shell_pod_name(shell_pod_name)
        # Calling exec and waiting for response
        exec_command = [
            '/bin/bash',
            '-c',
            cmd]
        resp = stream(self.v1api.connect_get_namespaced_pod_exec,
                      shell_name,
                      self.namespace,
                      command=exec_command,
                      stderr=True, stdin=False,
                      stdout=True, tty=False)
        self.logger.info("Response: " + resp)    
    

In [5]:
# nuclio: end-code

## Test
> This test uses the metrics data, created by the [Generator function](https://github.com/mlrun/demo-network-operations/blob/master/notebooks/generator.ipynb) from MLRun's [Network Operations Demo](https://github.com/mlrun/demo-network-operations)  
To test it yourself, please generate this dataset or use any of your available csv/parquet datasets.

In [6]:
from mlrun import code_to_function, mount_v3io, NewTask, mlconf, run_local
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'


### Define the execute test task

In [7]:
execute_task = NewTask(name='submit',
                         project='submit-proj',
                         params={'cmd':"echo ",'shell_pod_name':'shell'},                         
                         #params={'cmd':'ls -la /User/parquez/sh/'},     
                         handler=handler)

In [8]:
#submit_run = run_local(submit_task)

### Test on cluster

Convert the code to an MLRun function

In [9]:
fn = code_to_function('submit', handler='handler')
fn.spec.service_account='mlrun-api'
fn.apply(mount_v3io())
fn.export('function.yaml')

[mlrun] 2020-06-10 11:21:11,015 function spec saved to path: function.yaml


In [10]:
execute_run = fn.run(execute_task)

[mlrun] 2020-06-10 11:21:11,024 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-06-10 11:21:11,044 starting run submit uid=440152adda134c5fb0d50075020bfce2  -> http://10.192.248.103:8080
[mlrun] 2020-06-10 11:21:11,211 Job is running in the background, pod: submit-bphng
[mlrun] 2020-06-10 11:21:17,493 artifact path is not defined or is local, artifacts will not be visible in the UI
[mlrun] 2020-06-10 11:21:17,568 Starting to cmd
[mlrun] 2020-06-10 11:21:17,568 Running command cd /User/parquez/sh;ls -la;bash ./parquetinizer.sh faker '3 hours' '21 hours' 'h' 'parquez' 'default' 'Parquet' '6' 'k8s'
[mlrun] 2020-06-10 11:21:17,569 using in-cluster config.
[mlrun] 2020-06-10 11:21:17,784 10.200.0.6	default-tenant	avi-shell-6d9c467597-89lss
[mlrun] 2020-06-10 11:21:18,778 Response: total 0
-rwxr-xr-x 1 50 nogroup    327 May 20 13:24 alter_kv_view.sh
-rwxr-xr-x 1 50 nogroup   1562 May 20 13:24 hive_partition.sh
-rwxr-xr-x 1 50 nogroup    212 May 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
submit-proj,...020bfce2,0,Jun 10 11:21:17,completed,submit,host=submit-bphngkind=jobowner=iguaziov3io_user=iguazio,,cmd=cd /User/parquez/sh;ls -la;bash ./parquetinizer.sh faker '3 hours' '21 hours' 'h' 'parquez' 'default' 'Parquet' '6' 'k8s'shell_pod_name=shell,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 440152adda134c5fb0d50075020bfce2 --project submit-proj , !mlrun logs 440152adda134c5fb0d50075020bfce2 --project submit-proj
[mlrun] 2020-06-10 11:21:20,510 run executed, status=completed


### Show results